---
Load CSVs containing Iris training/test data into a TensorFlow Dataset.

Construct a neural network classifier.

Train the model using the training data.

Evaluate the accuracy of the model.

Classify new samples.

---

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from six.moves.urllib.request import urlopen

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [2]:
if not os.path.exists(IRIS_TRAINING):
  raw = urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'wb') as f:
    f.write(raw)

if not os.path.exists(IRIS_TEST):
  raw = urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'wb') as f:
    f.write(raw)

In [3]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [4]:
# Constructing the computation graph

# Specify feature type. 4 features per example. model_dir is where to save checkpoints data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]
# build net with 10,20 and 10 units
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                       hidden_units=[10,20, 20, 10],
                                       n_classes=3,
                                       model_dir="/tmp/iris_model1")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E87388FA58>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
print(training_set.data.shape)
print(training_set.target.shape)
# describe training input pipeline
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True
    )

(120, 4)
(120,)


In [25]:
# Fit the classifier to training data
classifier.train(input_fn=train_input_fn, steps=2000)
classifier.train(input_fn=train_input_fn, steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model1\model.ckpt.
INFO:tensorflow:loss = 212.094, step = 1
INFO:tensorflow:global_step/sec: 374.628
INFO:tensorflow:loss = 9.94038, step = 101 (0.286 sec)
INFO:tensorflow:global_step/sec: 442.028
INFO:tensorflow:loss = 9.77558, step = 201 (0.227 sec)
INFO:tensorflow:global_step/sec: 444.886
INFO:tensorflow:loss = 10.5664, step = 301 (0.218 sec)
INFO:tensorflow:global_step/sec: 470.599
INFO:tensorflow:loss = 7.89171, step = 401 (0.219 sec)
INFO:tensorflow:global_step/sec: 433.587
INFO:tensorflow:loss = 4.28263, step = 501 (0.219 sec)
INFO:tensorflow:global_step/sec: 476.439
INFO:tensorflow:loss = 6.61481, step = 601 (0.223 sec)
INFO:tensorflow:global_step/sec: 456.46
INFO:tensorflow:loss = 7.35095, step = 701 (0.218 sec)
INFO:tensorflow:global_step/sec: 430.492
INFO:tensorflow:loss = 6.81428, step = 801 (0.224 sec)
INFO:tensorflow:global_step/sec: 442.056
INFO:tensorflow:loss = 5.13218, 

In [26]:
# Evaluate model accuracy
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\n Test accuracy: " + str(accuracy_score) + "\n")

INFO:tensorflow:Starting evaluation at 2017-12-25-20:17:31
INFO:tensorflow:Restoring parameters from /tmp/iris_model1\model.ckpt-3000
INFO:tensorflow:Finished evaluation at 2017-12-25-20:17:31
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.966667, average_loss = 0.0655351, global_step = 3000, loss = 1.96605

 Test accuracy: 0.966667



In [27]:
# Classifying new samples
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print("New Samples, Class Predictions: " + str(predicted_classes))


INFO:tensorflow:Restoring parameters from /tmp/iris_model1\model.ckpt-3000
New Samples, Class Predictions: [array([b'1'], dtype=object), array([b'2'], dtype=object)]
